# Introduction: Interferometry vs. photometry


Let's say we have a spherical star whose intensity map, projected into a 2D at a specific vieweing orientation, is defined as $I(x,y)$. Finding the total brightness of the star at that orientation is a sum over the visible surface of the star: 

 $$ F = \iint I(x,y)dx dy $$
 
 
 This problem of fiding the disk-integrated brightness of a star--the observable quantity of photometry--has been solved in the [starry package](https://arxiv.org/abs/1810.06559) in a rather elegant way that, among other things, permits a description of the information content of photometric data. This is done first by representing the surface map of a star using [spherical harmonics](https://en.wikipedia.org/wiki/Spherical_harmonics). If the 3d surface of a star is represented using spherical harmonic coefficients $\mathbf{y}$, then we can write the intensity map as:
 
$$I(x,y) = \mathbf{\tilde{y}}^\top(x,y) \ \mathbf{R} \ \mathbf{y}$$

where $\mathbf{\tilde{y}}^\top(x,y)$ is the spherical harmonic basis, $\mathbf{R}$ is the rotation matrix into the correct viewing orientation with the viewer at $+\infty$ along the z axis and $\mathbf{y}$ is the vector of spherical harmonic coefficients. Because spherical harmonics form an orthonormal basis on a unit sphere, *any* map can be represented using a sufficiently high order expansion in the spherical harmonics. This makes it a natural choice to represent the surface of a star.

The real achievement of starry was to find an analytic way of performing the surface integral where a star is represented using spherical harmonics. This makes it extremely fast to compute photometric observables--light curves-- as a star rotates or even as a planet occults it. 

In this notebook, I will attempt to show that it is possible to use the same elegant description of the surface of a star in terms of spherical harmonics to find analytic observables used in interferometry. Recall that interferometric observations record a quantity called the visibility. The van-Cittert Zernike theorem relates the intensity map of a star to its visibility using a different kind of double integral from the one for photometry--the Fourier transform:

$$ V(u,v) = \iint I(x,y) e^{i(ux + vy)} dxdy $$

Where $V$ is the visibility at baseline (u,v). This integral is similar but not identical to the one from photometry, and a similar intuition should be able to help solve it. First, we must find a way to take the Fourier transform of the surface map expressed as an expansion in spherical harmonics. 

$$ V(u,v) = \iint \mathbf{\tilde{y}}^\top(x,y) e^{i(ux + vy)} dxdy \ \ \mathbf{R} \ \mathbf{y}$$ 

where we can pull out $\mathbf{R} \ \mathbf{y}$ from the integral as they do not depend on x and y. Now, we must find the Fourier transform of the spherical harmonic basis. Although this might appear difficult, I have found that half the terms in $\mathbf{\tilde{y}}^\top(x,y)$ (those with l+m even) map perfectly onto a polynomial basis defined on a unit disk which has an analytic Fourier transform--the Zernikes. The other half have an analytic Fourier transform in terms of spherical Bessel functions. Let's split the spherical harmonic basis into two complementary subspaces, called the [hemispheric harmonics (HSH) and the complementary hemispheric harmonics (CHSH)](https://opg.optica.org/oe/fulltext.cfm?uri=oe-27-26-37180&id=423949):
$$ \mathbf{\tilde{y}} = \mathbf{\tilde{y}}_{\mathrm{HSH}} \cup \ \mathbf{\tilde{y}}_{\mathrm{CHSH}}$$
What we must do is find another change of basis matrix $\mathbf{A}$ for which

$$ \mathbf{y}_{\mathrm{HSH}} = \mathbf{A} \ \mathbf{{z}} $$

where $\mathbf{\tilde{z}}$ is the zernike basis. If we can find such a matrix $\mathbf{A}$, we can solve the Fourier integral as follows:

$$ V(u,v) =  \left((\mathbf{A}\widehat{\mathbf{\tilde{z}}})^\top + \widehat{\mathbf{\tilde{y}}}_{\mathrm{CHSH}}^\top \right)\mathbf{R}\mathbf{y}$$

where $\widehat{\mathbf{\tilde{z}}}$ is the Zernike solution vector containing Fourier transforms of each term in the Zernike basis and $\widehat{\mathbf{\tilde{y}}}_{\mathrm{CHSH}}$ is the CHSH solution vector containing Fourier transforms of each CHSH term. In practice we do not do this all in the Cartesian basis, but use $V(\rho, \phi)$ instead of $V(u, v)$ and $r, \theta$ instead of $x, y$, allowing us to express the Zernike and spherical harmonic bases as separable products of Zernike polynomials and Legendre functions times cos $m \phi$ or sin $m \phi$ respectively. 

In [2]:
import sympy as sp
from sympy import symbols, sin, cos, Matrix, Eq, Rational, floor, sqrt
from sympy import simplify, factorial, pi, binomial, factor, expand, collect, gamma
from sympy.functions.special.tensor_functions import KroneckerDelta
from sympy import init_printing
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pandas as pd
from sympy import latex
from scipy.optimize import curve_fit
from IPython.display import display, Math

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Lets show terms in the zernike basis $\mathbf{\tilde{z}}(\rho,\phi)$ up to N of 5:

In [3]:
r, theta = sp.symbols('r, theta')

def zernike(j, r, theta):
    """ returns the jth term of the zernike polynomial basis"""
    n = int(np.ceil((-3 + np.sqrt(9 + 8 * j)) / 2))
    m = int(2 * j - n * (n + 2))
    res = 0
    for s in range(0,int((n-np.abs(m))/2)+1):
        res += Rational((-1)**s * factorial(n-s) / (factorial(s) * factorial((n+np.abs(m))/2 -s) * factorial((n-np.abs(m))/2 - s))) * r**(n-2*s)
    if m<0:
        return res*sin(-m*theta)
    elif m>0:
        return res*cos(m*theta)
    else:
        return res
    
jmax = lambda nmax: (nmax**2+3*nmax)//2
zbasis = Matrix([zernike(j, r, theta) for j in range(jmax(5)+1)])
zbasis


Matrix([
[                                   1],
[                        r*sin(theta)],
[                        r*cos(theta)],
[                   r**2*sin(2*theta)],
[                          2*r**2 - 1],
[                   r**2*cos(2*theta)],
[                   r**3*sin(3*theta)],
[           (3*r**3 - 2*r)*sin(theta)],
[           (3*r**3 - 2*r)*cos(theta)],
[                   r**3*cos(3*theta)],
[                   r**4*sin(4*theta)],
[      (4*r**4 - 3*r**2)*sin(2*theta)],
[                 6*r**4 - 6*r**2 + 1],
[      (4*r**4 - 3*r**2)*cos(2*theta)],
[                   r**4*cos(4*theta)],
[                   r**5*sin(5*theta)],
[      (5*r**5 - 4*r**3)*sin(3*theta)],
[(10*r**5 - 12*r**3 + 3*r)*sin(theta)],
[(10*r**5 - 12*r**3 + 3*r)*cos(theta)],
[      (5*r**5 - 4*r**3)*cos(3*theta)],
[                   r**5*cos(5*theta)]])

Now, let's show the hemispherical harmonic basis up to lmax of 5:

**IMPORTANT** So that I can demonstrate the results compactly, I am omitting a normalization factor A(l,m) from the hemispherical harmonic basis

In [4]:
def hsh(n, r, theta):
    """Returns the nth term of the hemispheric harmonic basis (half of the spherical harmonics)

    Args:
        n (int): unrolled index identical to the OSI/ANSI indexing scheme of the Zernikes
        r (sympy variable/float): variable for the radial coordinate, can be a float or a sympy variable (?)

    Returns:
        expr: sympy expression with the nth term of the basis
    """
    l = int(np.ceil((-3 + np.sqrt(9 + 8 * n)) / 2))
    m = int(2 * n - l * (l + 2))
    if m<0:
        angular = sin(-m*theta)
    elif m>0:
        angular = cos(m*theta)
    else:
        angular = 1
    m = np.abs(m)

    res = 0
    for k in range(0,l-m+1):
        for j in range(0,int(k/2)+1):
            res += 2**l * (gamma(Rational(l+m+k-1, 2)+1)/(factorial(k) * factorial(l-m-k) * gamma(Rational(-l+m+k-1, 2)+1))) * binomial(Rational(k,2),j) * (-1)**j * r**(2*j+m)
    return res*angular*(-1)**(l)

hshbasis = Matrix([hsh(n, r, theta) for n in range(jmax(5)+1)])
hshbasis
    

Matrix([
[                                           1],
[                               -r*sin(theta)],
[                               -r*cos(theta)],
[                         3*r**2*sin(2*theta)],
[                                1 - 3*r**2/2],
[                         3*r**2*cos(2*theta)],
[                       -15*r**3*sin(3*theta)],
[              -(-15*r**3/2 + 6*r)*sin(theta)],
[              -(-15*r**3/2 + 6*r)*cos(theta)],
[                       -15*r**3*cos(3*theta)],
[                       105*r**4*sin(4*theta)],
[        (-105*r**4/2 + 45*r**2)*sin(2*theta)],
[                      35*r**4/8 - 5*r**2 + 1],
[        (-105*r**4/2 + 45*r**2)*cos(2*theta)],
[                       105*r**4*cos(4*theta)],
[                      -945*r**5*sin(5*theta)],
[      -(-945*r**5/2 + 420*r**3)*sin(3*theta)],
[-(315*r**5/8 - 105*r**3/2 + 15*r)*sin(theta)],
[-(315*r**5/8 - 105*r**3/2 + 15*r)*cos(theta)],
[      -(-945*r**5/2 + 420*r**3)*cos(3*theta)],
[                      -945*r**

Now, I define an intermediate basis, which consists of a monomial times an angular factor of either $sin$ or $cos(m \theta)$. This basis, which I'm calling the polynomial basis, is indexed in the same way as the zernikes or hemispheric harmonics, but serves as an intermediate.

In [5]:
def poly(j, r, theta):
    """ returns the jth term of the zernike polynomial basis"""
    n = int(np.ceil((-3 + np.sqrt(9 + 8 * j)) / 2))
    m = int(2 * j - n * (n + 2))
    res = r**n
    if m<0:
        return res*sin(-m*theta)
    elif m>0:
        return res*cos(m*theta)
    else:
        return res
    
jmax = lambda nmax: (nmax**2+3*nmax)//2
polybasis = Matrix([poly(j, r, theta) for j in range(jmax(5)+1)])
polybasis

Matrix([
[                1],
[     r*sin(theta)],
[     r*cos(theta)],
[r**2*sin(2*theta)],
[             r**2],
[r**2*cos(2*theta)],
[r**3*sin(3*theta)],
[  r**3*sin(theta)],
[  r**3*cos(theta)],
[r**3*cos(3*theta)],
[r**4*sin(4*theta)],
[r**4*sin(2*theta)],
[             r**4],
[r**4*cos(2*theta)],
[r**4*cos(4*theta)],
[r**5*sin(5*theta)],
[r**5*sin(3*theta)],
[  r**5*sin(theta)],
[  r**5*cos(theta)],
[r**5*cos(3*theta)],
[r**5*cos(5*theta)]])

## Change of basis matrix:

If I can write the basis change from zernikes to the polynomial basis as $\mathbf{A}_{\mathrm{Z} \rightarrow \mathrm{P}}$, and from HSH to polynomials as $\mathbf{A}_{\mathrm{HSH} \rightarrow \mathrm{P}}$, then the full basis change is just:

$$\mathbf{A} = \mathbf{A}_{\mathrm{Z} \leftarrow \mathrm{P}} \ \mathbf{A}_{\mathrm{HSH} \rightarrow \mathrm{P}}$$

where $\mathbf{A}_{\mathrm{Z} \leftarrow \mathrm{P}} = \mathbf{A}_{\mathrm{Z} \rightarrow \mathrm{P}}^{-1}$

Now lets write the code to create each of the intermediate basis change matrices $\mathbf{A}_{\mathrm{Z} \rightarrow \mathrm{P}}$ and $\mathbf{A}_{\mathrm{HSH} \rightarrow \mathrm{P}}$

In [6]:
def j_to_nm(j):
    n = int(np.ceil((-3 + np.sqrt(9 + 8 * j)) / 2))
    m = int(2 * j - n * (n + 2))
    return n,m
def nm_to_j(n,m):
    return int((n*(n+2)+m)/2)

jmax = lambda nmax: (nmax**2+3*nmax)//2

def z_to_p(j, jmax):
    """ returns the jth term of the zernike polynomial basis"""
    n,m = j_to_nm(j)
    res = np.zeros(jmax+1)
    for s in range(0,int((n-np.abs(m))/2)+1):
        #this particular term of the polynomial has r^(n-2*s), that means we drop the r and instead add this to the array for n=n-2*s and m=m
        res[nm_to_j(int(n-2*s), m)] += (-1)**s * factorial(n-s) / (factorial(s) * factorial((n+np.abs(m))/2 -s) * factorial((n-np.abs(m))/2 - s))
    return res

def hsh_to_p(n, nmax):
    """ returns the jth term of the HSH to polynomial basis"""
    l,m = j_to_nm(n)
    res = np.zeros(nmax+1)
    for k in range(0,l-np.abs(m)+1):
        for j in range(0,int(k/2)+1):
            #this particular term of the polynomial has r^(2*j+np.abs(m)), that means we drop the r and instead add this to the array for n=n-2*s and m=m
            res[nm_to_j(int(2*j+np.abs(m)),m)] += (-1)**(l) * 2**l * (gamma(Rational(l+np.abs(m)+k-1, 2)+1)/(factorial(k) * factorial(l-np.abs(m)-k) * gamma(Rational(-l+np.abs(m)+k-1, 2)+1))) * binomial(Rational(k,2),j) * (-1)**j
    return res
    
display(Math(latex(zernike(11, r, theta))))
display(Math(latex(z_to_p(11,jmax(4)))))
display(Math(latex(hsh(2, r, theta))))
display(Math(latex(hsh_to_p(2,jmax(4)))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
def A_z_to_p(jmax):
    A = np.zeros((jmax+1,jmax+1))
    for j in range(jmax+1):
        A[j] = z_to_p(j, jmax)
    return A

def A_hsh_to_p(jmax):
    A = np.zeros((jmax+1,jmax+1))
    for j in range(jmax+1):
        A[j] = hsh_to_p(j, jmax)
    return A

display(Math(latex(Matrix(A_z_to_p(jmax(5))))))

display(Math(latex(Matrix(A_hsh_to_p(jmax(5))))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Now, we can put together the final matrix A!

In [8]:
display(Math(latex(hsh(7, r, theta))))
zbasis = Matrix([zernike(j, r, theta) for j in range(jmax(5)+1)])
hshbasis = Matrix([hsh(j, r, theta) for j in range(jmax(5)+1)])
hsh_vector = np.zeros(jmax(5)+1)
hsh_vector[7] = 1
A = Matrix(np.linalg.inv(A_z_to_p(jmax(5)))).T*Matrix(A_hsh_to_p(jmax(5))).T
display(Math(latex(A)+latex(Matrix(hsh_vector))+"="+latex(A*Matrix(hsh_vector))))
display(Math(latex(sp.simplify((A*Matrix(hsh_vector)).T*zbasis))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>